<a href="https://colab.research.google.com/github/seki-shu/Group15_Multimodal/blob/Dreamer-V2/Dreamer_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

## libraryのimport

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.distributions as td
from torch.distributions import Normal, Categorical, OneHotCategorical, OneHotCategoricalStraightThrough, Bernoulli
from torch.distributions import kl_divergence

## deviceの確認

In [ ]:
# torch.deviceを定義．この変数は後々モデルやデータをGPUに転送する時にも使います
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# モデルの実装


## RSSM

In [ ]:
class RSSM(nn.Module):
    """
    RSSMのクラス.

    Methods:
    --------
        recurrent: 決定的状態hをrnnで得る.
        get_prior: 確率的状態zの事前分布を得る.
        get_posterior: 確率的状態zの事後分布を得る.
    """
    def __init__(self, mlp_hidden_dim: int, h_dim: int, z_dim: int, a_dim: int, n_classes: int, embedding_dim: int):
        """
        コンストラクタ.

        Params:
        --------
            mlp_hidden_dim : int
               mlpに通した後の次元数.
            h_dim : int
               決定的状態hの次元数.
            z_dim : int
               確率的状態zの次元数.
            a_dim : int
               行動の次元数.
            n_classes : int
               確率的状態zのカテゴリ数.
            embedding_dim : int
               観測画像の埋め込み次元数.
        """
        super().__init__()

        self.h_dim = h_dim
        self.z_dim = z_dim
        self.a_dim = a_dim
        self.n_classes = n_classes

        # Recurrent model
        # h_t = f(h_t-1, z_t-1, a_t-1)
        self.z_a_hidden = nn.Linear(z_dim * n_classes + a_dim, mlp_hidden_dim)
        self.rnn = nn.GRUCell(mlp_hidden_dim, h_dim)

        # Prior prediction
        # z_t+1_hat = f(h_t+1)
        self.prior_hidden = nn.Linear(h_dim, mlp_hidden_dim)
        self.prior_logits = nn.Linear(mlp_hidden_dim, z_dim * n_classes)

        # Posterior
        # z_t+1 = f(h_t+1, o_t+1)
        self.posterior_hidden = nn.Linear(h_dim + embedding_dim, mlp_hidden_dim)
        self.posterior_logits = nn.Linear(mlp_hidden_dim, z_dim * n_classes)

    def recurrent(self, h_prev: torch.Tensor, z_prev: torch.Tensor, a_prev: torch.Tensor, rnn_hidden: torch.Tensor):
        """
        決定的状態の状態遷移を求めるメソッド. RNNとしてGRUを使用する.
        h_t+1 = f(h_t, z_t, a_t)

        Params:
        -------
            h_prev : torch.Tensor (batch size, h_dim)
                現在時刻の決定的状態h_t
            z_prev : torch.Tensor (batch size, z_dim)
                現在時刻の確率的状態z_t
            a_prev : torch.Tensor (batch size, a_dim)
                現在時刻のアクションa_t

        Returns:
        -------
            h : torch.Tensor (batch size, h_dim)
                次時刻の決定的状態h_t+1
        """
        mlp_hidden = F.elu(self.z_a_hidden(torch.cat([z_prev, a_prev], dim=1)))
        h = self.rnn(mlp_hidden, rnn_hidden)
        return h

    def get_prior(self,h: torch.Tensor, detach = False):
        """
        確率的状態zの事前分布を求めるメソッド.
        z_t+1_hat ~ p(h_t+1)

        Params:
            h : torch.Tensor (batch size, h_dim)
                次時刻の決定的状態h_t+1.
            detach : bool
                Trueの場合、detachする.

        ---------
        Returns:
            prior : torch.distributions.Distribution
                事前分布を求めるためのカテゴリカル分布.
            detached_prior : torch.distributions.Independent
                detachされた事前分布を求めるためのカテゴリカル分布.
        """
        mlp_hidden = F.elu(self.prior_hidden(h))
        logits = self.prior_logits(mlp_hidden) # (B, z_dim * n_classes,)
        logits = logits.reshape(logits.shape[0], self.z_dim, self.n_classes) # (B, z_dim, n_classes)

        prior = td.Independent(OneHotCategoricalStraightThrough(logits=logits), 1)
        if detach:
            detached_prior = td.Independent(OneHotCategoricalStraightThrough(logits=logits.detach()), 1)
            return prior, detached_prior
        return prior

    def get_posterior(self, h: torch.Tensor, embedded_obs: torch.Tensor, detach = False):
        """
        確率的状態zの事後分布を求めるメソッド.
        z_t+1 ~ p(h_t+1, o_t+1)

        Params:
        -------
            h : torch.Tensor (batch size, h_dim)
                次時刻の決定的状態h_t+1.
            embedded_obs : torch.Tensor (batch size, embedding_dim)
                次時刻の画像の埋め込み
            detach : bool
                Trueの場合、detachする.

        Returns:
        --------
            posterior : torch.distributions.Distribution
                事後分布を求めるためのカテゴリカル分布.
            detached_posterior : torch.distributions.Independent
                detachされた事後分布を求めるためのカテゴリカル分布.
        """
        mlp_hidden = F.elu(self.posterior_hidden(torch.cat([h, embedded_obs], dim=1)))
        logits = self.posterior_logits(mlp_hidden) # (B, z_dim * n_classes)
        logits = logits.reshape(logits.shape[0], self.z_dim, self.n_classes) # (B, z_dim, n_classes)

        posterior = td.Independent(OneHotCategoricalStraightThrough(logits=logits), 1)
        if detach:
            detached_posterior = td.Independent(OneHotCategoricalStraightThrough(logits=logits.detach()), 1)
            return posterior, detached_posterior
        return posterior

## Encoder

In [ ]:
class Encoder(nn.Module):
    """
    画像(3, 64, 64)をエンコードする.

    Methods:
    -------
        forward: 観測画像を埋め込む.
    """
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2)

    def forward(self, obs: torch.Tensor) -> torch.Tensor:
        """
        順伝播を行うメソッド．観測画像をベクトルに埋め込む．
        (3, 64, 64) -> (1024, ) にエンコード

        Params:
        -------
            obs : torch.Tensor (batch size, 3, 64, 64)
                環境から得られた観測画像．

        Returns:
        -------
            embed : torch.Tensor (batch size, 1024)
                観測を1024次元のベクトルに埋め込んだもの．
        """
        embed = F.elu(self.conv1(obs))
        embed = F.elu(self.conv2(embed))
        embed = F.elu(self.conv3(embed))
        embed = F.elu(self.conv4(embed)).reshape(embed.shape[0], -1)
        return embed

## Decoder

In [ ]:
class Decoder(nn.Module):
    """
    内部状態表現h, zから画像(3, 64, 64)を再構成する.
    ただし画像は平均を計算して、その平均で分散１の正規分布から得る.

    Methods:
        forward: 画像の再構成を行う.
    """
    def __init__(self, h_dim: int, z_dim: int, n_classes: int):
        """
        コンストラクタ．

        Params:
        --------
            h_dim : int
                決定的状態hの次元数．
            z_dim : int
                確率的状態zの次元数．
            n_classes : int
                確率的状態zのカテゴリ数．
        """
        super().__init__()
        self.fc = nn.Linear(h_dim + z_dim * n_classes, 1024)
        self.deconv1 = nn.ConvTranspose2d(1024, 128, kernel_size=5, stride=2)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2)
        self.deconv3 = nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2)
        self.deconv4 = nn.ConvTranspose2d(32, 3, kernel_size=6, stride=2)

    def forward(self, h: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        """
        順伝播を行うメソッド．内部状態h, zから画像を再構成する．
        mean = f(h_t, z_t)
        o_t = N(mean, 1)

        Params:
        --------
            h : torch.Tensor (batch size, h_dim)
                決定的状態h
            z : torch.Tensor (batch size, z_dim * n_classes)
                確率的状態z

        Returns:
        -------
            obs_dist : torch.distributions.Independent
                再構成された画像を得るための多変量正規分布．
        """
        x = self.fc(torch.cat([h, z], dim=1))
        x = x.reshape(x.shape[0], 1024, 1, 1)
        x = F.elu(self.deconv1(x))
        x = F.elu(self.deconv2(x))
        x = F.elu(self.deconv3(x))
        mean = self.deconv4(x)
        obs_dist = td.Independent(Normal(mean, 1), 3)
        return obs_dist


## Reward Model

In [ ]:
class RewardModel(nn.Module):
    """
    報酬予測モデル.
    ただし報酬は平均を計算して、その平均で分散が1の正規分布から得る.

    Methods:
    -------
        forward: 報酬予測を行う.
    """
    def __init__(self, h_dim: int, z_dim: int, n_classes: int, hidden_dim: int):
        """
        コンストラクタ.

        Params:
        -------
            h_dim : int
                決定的状態hの次元数.
            z_dim : int
                確率的状態zの次元数.
            n_classes : int
                確率的状態zのカテゴリ数.
            hidden_dim : int
                MLPを通した後のユニット数.

        """
        super().__init__()
        self.fc1 = nn.Linear(h_dim + z_dim * n_classes, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)

    def forward(self, h: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        """
        報酬予測を行うメソッド.
        mean = f(h_t, z_t)
        r_t = N(mean, 1)

        Params:
        ----------
            h : torch.Tensor (batch size, h_dim)
                決定的状態h
            z : torch.Tensor (batch size, z_dim * n_classes)
                確率的状態z

        Returns:
        -------
            reward_dist : torch.distributions.Independent
                報酬予測をするための多変量正規分布.
        """
        x = F.elu(self.fc1(torch.cat([h, z], dim=1)))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        mean = self.fc4(x)
        reward_dist = td.Independent(Normal(mean, 1), 1)
        return reward_dist

## Discount Model


In [ ]:
class DiscountModel(nn.Module):
    """
    現在のエピソードが終端かどうかを求めるモデル.

    Methods:
    --------
        forward: 終端かどうか予測を行う.
    """
    def __init__(self, h_dim: int, z_dim: int, n_classes: int, hidden_dim: int):
        """
        コンストラクタ.

        Params:
        -------
            h_dim : int
                決定的状態hの次元数.
            z_dim : int
                確率的状態zの次元数.
            n_classes : int
                確率的状態zのカテゴリ数.
            hidden_dim : int
                MLPを通した後のユニット数.
        """
        super().__init__()
        self.fc1 = nn.Linear(h_dim + z_dim * n_classes, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 1)

    def forward(self, h: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        """
        現在のエピソードが終端かどうか求める.
        p = f(h_t, z_t)
        gamma_t = Bernoulli(p)

        Params:
        -------
            h : torch.Tensor (batch size, h_dim)
                決定的状態h
            z : torch.Tensor (batch size, z_dim * n_classes)
                確率的状態z

        Returns:
        -------
            discount_dist : torch.distributions.Independent
                終端かどうか予測を行うベルヌーイ分布
        """
        x = F.elu(self.fc1(torch.cat([h, z], dim=1)))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        p_logits = self.fc4(x)
        discount_dist = td.Independent(Bernoulli(logits=p_logits), 1)
        return discount_dist

## Actor

In [ ]:
class Actor(nn.Module):
    """
    最適行動を出力するモデル.
    今回はAtariではないので連続な行動を出力する.

    Methods:
    -------
        forward: 最適行動を出力する.
    """
    def __init__(self, action_dim: int, h_dim: int, z_dim: int, n_classes: int, hidden_dim: int):
        """
        コンストラクタ.

        Params:
        -------
            action_dim : int
                行動の次元数.
            h_dim : int
                決定的状態hの次元数.
            z_dim : int
                確率的状態zの次元数.
            n_classes : int
                確率的状態zのカテゴリ数.
            hidden_dim : int
                MLPを通した後のユニット数.
        """
        super().__init__()
        self.fc1 = nn.Linear(h_dim + z_dim * n_classes, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim, action_dim)
        self.fc_std = nn.Linear(hidden_dim, action_dim)

    def forward(self, h: torch.Tensor, z: torch.Tensor, eval: bool):
        """
        最適行動を出力する.
        a_t = f(h_t, z_t)

        Params:
        -------
            h : torch.Tensor (batch size, h_dim)
                決定的状態h.
            z : torch.Tensor (batch size, z_dim * n_classes)
                確率的状態z.
            eval : bool
                評価モードかどうか.

        Returns:
        -------
            actions : torch.Tensor (batch size, action_dim)
                最適行動.
            action_log_probs : torch.Tensor (batch size, 1)
                最適行動の対数尤度.
            action_entropy : torch.Tensor (batch size, 1)
                最適行動のエントロピー.
        """
        x = F.elu(self.fc1(torch.cat([h, z], dim=1)))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        mean = self.fc_mean(x)
        std = F.softplus(self.fc_std(x))

        if eval:
            actions = torch.tanh(mean) # action spaceを(-1, 1)にしてる. 環境によっては変更必要かも.
            return actions, None, None

        # Dreamer V2の連続空間でのActorがどうなっているかいまいち分からなかったので、とりあえず以下のように実装しました。
        # まずactionを正規分布からサンプルした後、tanh関数に通して(-1, 1)にします。
        # 対数尤度を求めるときに、log p(actions) = log Normal(unscaled_actions) * |det(∂unscaled_actions / ∂actions)|(https://arxiv.org/pdf/1801.01290 Appendix C)となります
        # tanhの微分は1-tanh^2で、正規分布の分散共分散行列は対角行列なので、ヤコビアンはdiag(1 - tanh^2)となります。
        # 間違ってるかもしれません...
        action_dist = td.Independent(Normal(mean, std), 1)
        unscaled_actions = action_dist.rsample()
        actions = torch.tanh(unscaled_actions)

        action_log_probs = action_dist.log_prob(unscaled_actions) - torch.log(1 - actions.pow(2) + 1e-7).sum(dim=1, keepdim=True)
        action_entropy = action_dist.entropy()
        return actions, action_log_probs, action_entropy